<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/app_single_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# app_single.py
# A complete end-to-end single-file platform: FastAPI backend, OAuth, dashboard UI,
# multimodal AI suite with demo fallbacks, blob storage, Γ-universe simulation, metrics,
# and a typed client. Designed for local-first operation and production hardening.

import os
import io
import json
import time
import base64
import hashlib
import secrets
import logging
import datetime as dt
from typing import List, Optional, Dict, Any, Tuple
from urllib.parse import urlencode

from fastapi import FastAPI, Depends, File, UploadFile, Form, HTTPException, status, Request, Response, Body, Query
from fastapi.responses import HTMLResponse, JSONResponse, RedirectResponse, PlainTextResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from pydantic import BaseModel, BaseSettings, Field, AnyUrl
from starlette.middleware.sessions import SessionMiddleware
from starlette.background import BackgroundTasks

# Optional heavy deps are imported lazily in production paths
# NLP: transformers; Vision: ultralytics; Speech: whisper
import httpx
from itsdangerous import URLSafeSerializer

try:
    from rich.logging import RichHandler
    RICH = True
except Exception:
    RICH = False

try:
    import numpy as np
    from PIL import Image
except Exception:
    np = None
    Image = None


# ---------------------------
# Settings and configuration
# ---------------------------

class Settings(BaseSettings):
    DEMO_MODE: bool = Field(default=True)
    SECRET_KEY: str = Field(default="change-me")
    SESSION_COOKIE_NAME: str = Field(default="session")
    FASTAPI_BASE_URL: str = Field(default="http://127.0.0.1:8000")
    BLOB_DIR: str = Field(default="./blobs")
    MAX_UPLOAD_MB: int = Field(default=50)

    # OAuth
    OAUTH_REDIRECT_BASE: Optional[str] = None  # e.g., http://127.0.0.1:8000
    GOOGLE_CLIENT_ID: Optional[str] = None
    GOOGLE_CLIENT_SECRET: Optional[str] = None
    GITHUB_CLIENT_ID: Optional[str] = None
    GITHUB_CLIENT_SECRET: Optional[str] = None
    MS_CLIENT_ID: Optional[str] = None
    MS_CLIENT_SECRET: Optional[str] = None

    # Models
    NLP_MODEL: str = Field(default="google/flan-t5-base")
    YOLO_MODEL: str = Field(default="yolov8n.pt")
    WHISPER_MODEL: str = Field(default="base")

    class Config:
        env_file = ".env"
        case_sensitive = False

settings = Settings()

# ---------------------------
# Logging
# ---------------------------

LOG_LEVEL = os.getenv("LOG_LEVEL", "INFO").upper()
logging.basicConfig(
    level=getattr(logging, LOG_LEVEL, logging.INFO),
    format="%(asctime)s %(levelname)s %(name)s - %(message)s",
    handlers=[RichHandler(markup=True)] if RICH else None,
)
log = logging.getLogger("platform")

# ---------------------------
# App init
# ---------------------------

app = FastAPI(title="Multimodal AI + Γ-Universe Platform", version="1.0.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)
app.add_middleware(SessionMiddleware, secret_key=settings.SECRET_KEY, session_cookie=settings.SESSION_COOKIE_NAME)

# Templates in-memory (single-file). We'll mount a dyn loader.
templates = Jinja2Templates(directory=".")

# Ensure blob dir
os.makedirs(settings.BLOB_DIR, exist_ok=True)

# URL signer for blob download tokens
signer = URLSafeSerializer(settings.SECRET_KEY, salt="blob-url")

# ---------------------------
# In-memory stores
# ---------------------------

ACTIVITY: List[Dict[str, Any]] = []
METRICS: Dict[str, Any] = {
    "start_time": time.time(),
    "requests_total": 0,
    "errors_total": 0,
    "nlp_calls": 0,
    "vision_calls": 0,
    "speech_calls": 0,
    "blob_uploads": 0,
    "sim_runs": 0,
}

def record(event: str, detail: Dict[str, Any] = None, user: Optional[str] = None):
    entry = {
        "ts": dt.datetime.utcnow().isoformat() + "Z",
        "event": event,
        "detail": detail or {},
        "user": user,
        "demo": settings.DEMO_MODE,
    }
    ACTIVITY.append(entry)
    if len(ACTIVITY) > 1000:
        del ACTIVITY[:200]
    log.info(f"{event}: {detail}")

# ---------------------------
# OAuth setup
# ---------------------------

# Minimal OAuth via manual endpoints to avoid extra deps
OAUTH_PROVIDERS = {
    "google": {
        "auth_url": "https://accounts.google.com/o/oauth2/v2/auth",
        "token_url": "https://oauth2.googleapis.com/token",
        "userinfo_url": "https://openidconnect.googleapis.com/v1/userinfo",
        "scope": "openid email profile",
        "client_id": settings.GOOGLE_CLIENT_ID,
        "client_secret": settings.GOOGLE_CLIENT_SECRET,
    },
    "github": {
        "auth_url": "https://github.com/login/oauth/authorize",
        "token_url": "https://github.com/login/oauth/access_token",
        "userinfo_url": "https://api.github.com/user",
        "scope": "read:user user:email",
        "client_id": settings.GITHUB_CLIENT_ID,
        "client_secret": settings.GITHUB_CLIENT_SECRET,
    },
    "microsoft": {
        "auth_url": "https://login.microsoftonline.com/common/oauth2/v2.0/authorize",
        "token_url": "https://login.microsoftonline.com/common/oauth2/v2.0/token",
        "userinfo_url": "https://graph.microsoft.com/oidc/userinfo",
        "scope": "openid email profile",
        "client_id": settings.MS_CLIENT_ID,
        "client_secret": settings.MS_CLIENT_SECRET,
    },
}

def oauth_enabled(provider: str) -> bool:
    cfg = OAUTH_PROVIDERS.get(provider)
    return bool(cfg and cfg["client_id"] and cfg["client_secret"] and settings.OAUTH_REDIRECT_BASE)

def get_user(request: Request) -> Optional[Dict[str, Any]]:
    return request.session.get("user")

def require_user(request: Request) -> Dict[str, Any]:
    user = get_user(request)
    if not user:
        raise HTTPException(status_code=401, detail="Unauthorized")
    return user

# ---------------------------
# Utilities
# ---------------------------

def file_size_ok(upload: UploadFile):
    # Not perfect for streams; for demo we allow check via header if present
    return True

def now_iso():
    return dt.datetime.utcnow().isoformat() + "Z"

def b64png(image_bytes: bytes) -> str:
    return "data:image/png;base64," + base64.b64encode(image_bytes).decode("utf-8")

def safe_int(v, d=0):
    try:
        return int(v)
    except Exception:
        return d

# ---------------------------
# Models and schemas
# ---------------------------

class Health(BaseModel):
    status: str
    uptime_seconds: float
    demo_mode: bool
    version: str

class NLPRequest(BaseModel):
    task: str = Field(description="summarize|translate|classify|generate")
    input_text: str

class NLPResponse(BaseModel):
    task: str
    output_text: str
    model: str
    demo: bool
    latency_ms: int

class VisionResponse(BaseModel):
    detections: List[Dict[str, Any]]
    model: str
    demo: bool
    latency_ms: int

class SpeechResponse(BaseModel):
    text: str
    model: str
    demo: bool
    latency_ms: int

class BlobUploadResponse(BaseModel):
    name: str
    size: int
    url: str
    demo: bool

class SimRequest(BaseModel):
    seed: Dict[str, Any]
    steps: int = 10
    novelty_limit: float = 0.3
    coherence_floor: float = 0.5
    snapshot_every: int = 5

class SimSnapshot(BaseModel):
    step: int
    state: Dict[str, Any]
    provenance: List[Dict[str, Any]]
    metrics: Dict[str, float]

class SimResponse(BaseModel):
    final_state: Dict[str, Any]
    snapshots: List[SimSnapshot]
    laws_applied: List[str]
    demo: bool
    latency_ms: int

# ---------------------------
# Lazy-loaded model handles
# ---------------------------

class ModelHub:
    nlp_pipe = None
    yolo = None
    whisper = None

HUB = ModelHub()

def load_nlp():
    if HUB.nlp_pipe is not None:
        return HUB.nlp_pipe
    from transformers import pipeline
    HUB.nlp_pipe = {
        "summarize": pipeline("summarization", model=settings.NLP_MODEL),
        "generate": pipeline("text2text-generation", model=settings.NLP_MODEL),
        "translate": pipeline("translation_en_to_de", model="t5-small"),
        "classify": pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english"),
    }
    return HUB.nlp_pipe

def load_yolo():
    if HUB.yolo is not None:
        return HUB.yolo
    from ultralytics import YOLO
    HUB.yolo = YOLO(settings.YOLO_MODEL)
    return HUB.yolo

def load_whisper():
    if HUB.whisper is not None:
        return HUB.whisper
    import whisper
    HUB.whisper = whisper.load_model(settings.WHISPER_MODEL)
    return HUB.whisper

# ---------------------------
# Demo implementations
# ---------------------------

def demo_summarize(text: str) -> str:
    words = text.split()
    if len(words) <= 30:
        return text
    return " ".join(words[:25]) + " ..."

def demo_generate(text: str) -> str:
    return f"{text.strip()} — and therefore, the system evolves."

def demo_translate_en_de(text: str) -> str:
    # naive fake
    return f"[DEMO-DE] {text}"

def demo_classify(text: str) -> str:
    return "POSITIVE" if any(w in text.lower() for w in ["good", "great", "love", "excellent"]) else "NEUTRAL"

def demo_yolo(img: Image.Image) -> List[Dict[str, Any]]:
    # Simple color-block heuristic
    arr = np.array(img.resize((64, 64)))
    mean = arr.mean(axis=(0,1)).tolist()
    return [{"label": "scene", "confidence": 0.42, "box": [0,0,img.width,img.height], "mean_rgb": mean}]

def demo_whisper_dummy() -> str:
    return "Transcription unavailable in demo. This is a placeholder."

# ---------------------------
# Γ-universe simulation engine
# ---------------------------

# Laws Λ1–Λ9 are expressed as functions over state and provenance.
# We keep a compact but expressive model.

def coherence_score(state: Dict[str, Any]) -> float:
    # Heuristic: fewer contradictions and tighter value ranges -> higher score
    contradictions = len(state.get("_contradictions", []))
    spread = 0.0
    for k, v in state.items():
        if isinstance(v, (int, float)):
            spread += abs(float(v))
    base = 1.0 / (1.0 + contradictions + 0.01 * spread)
    return min(1.0, base)

def novelty_distance(prev: Dict[str, Any], nxt: Dict[str, Any]) -> float:
    keys = set(prev.keys()) | set(nxt.keys())
    dist = 0.0
    for k in keys:
        a = prev.get(k)
        b = nxt.get(k)
        if a == b:
            continue
        if isinstance(a, (int,float)) and isinstance(b, (int,float)):
            dist += min(1.0, abs(float(a)-float(b))/ (abs(float(a))+1e-6 + abs(float(b))+1e-6))
        else:
            dist += 0.5
    return min(1.0, dist / max(1, len(keys)))

def dialectic(a, b):
    # Simple synthesis: prefer specific over general, numbers averaged, text concatenated
    if isinstance(a, (int,float)) and isinstance(b, (int,float)):
        return (a + b) / 2.0
    if isinstance(a, str) and isinstance(b, str):
        if a in b:
            return b
        if b in a:
            return a
        return f"{a} | {b}"
    if isinstance(a, list) and isinstance(b, list):
        return list({json.dumps(x, sort_keys=True) for x in (a+b)})
    if isinstance(a, dict) and isinstance(b, dict):
        out = dict(a)
        for k, v in b.items():
            out[k] = dialectic(out.get(k), v) if k in out else v
        return out
    return b if b is not None else a

def apply_laws(state: Dict[str, Any], step: int, prov: List[Dict[str, Any]],
               novelty_limit: float, coherence_floor: float) -> Tuple[Dict[str, Any], List[str], Dict[str, float]]:
    applied = []
    s_prev = json.loads(json.dumps(state))  # deep copy

    # Λ1 Identity persistence: copy forward baseline
    s_next = {k: v for k, v in state.items() if not k.startswith("_")}
    applied.append("Λ1")

    # Λ2 Locality: propose local evolutions
    proposals = {}
    for k, v in s_next.items():
        if isinstance(v, (int, float)):
            proposals[k] = v + (0.1 if step % 2 == 0 else -0.08)
        elif isinstance(v, str) and len(v) < 120:
            proposals[k] = v if step % 3 else v + "·"
        else:
            proposals[k] = v
    applied.append("Λ2")

    # Λ3 Provenance conservation
    prov.append({"step": step, "from": s_prev, "proposal": proposals, "at": now_iso()})
    s_prop = proposals
    applied.append("Λ3")

    # Λ4 Dialectic synthesis for conflicts
    s_syn = {}
    for k in set(s_next.keys()) | set(s_prop.keys()):
        if k in s_next and k in s_prop:
            s_syn[k] = dialectic(s_next[k], s_prop[k])
        else:
            s_syn[k] = s_next.get(k, s_prop.get(k))
    applied.append("Λ4")

    # Λ5 Novelty constraint
    nov = novelty_distance(s_prev, s_syn)
    if nov > novelty_limit:
        # damp changes
        for k, v in s_syn.items():
            if isinstance(v, (int,float)) and isinstance(s_prev.get(k), (int,float)):
                s_syn[k] = (s_prev[k] + v) / 2.0
        applied.append("Λ5-clamped")
    else:
        applied.append("Λ5-ok")

    # Λ6 Coherence check
    coh = coherence_score(s_syn)
    if coh < coherence_floor:
        # revert risky keys
        for k in list(s_syn.keys()):
            s_syn[k] = s_prev.get(k, s_syn[k])
        applied.append("Λ6-revert")
    else:
        applied.append("Λ6-ok")

    # Λ7 Contradiction handling
    contradictions = []
    for k, v in s_syn.items():
        if isinstance(v, (int, float)) and abs(float(v)) > 1e6:
            contradictions.append({"k": k, "v": v, "reason": "magnitude"})
    if contradictions:
        s_syn["_contradictions"] = s_syn.get("_contradictions", []) + contradictions
    applied.append("Λ7")

    # Λ8 Echo archive flagging handled by caller via snapshots
    applied.append("Λ8")

    # Λ9 Law-of-laws tuning marker (no-ops here, but record meta)
    metrics = {"novelty": nov, "coherence": coherence_score(s_syn)}
    applied.append("Λ9")

    return s_syn, applied, metrics

def run_simulation(req: SimRequest) -> Tuple[Dict[str, Any], List[SimSnapshot], List[str]]:
    start = time.time()
    state = dict(req.seed)
    provenance: List[Dict[str, Any]] = []
    snapshots: List[SimSnapshot] = []
    laws_union: List[str] = []
    for step in range(1, req.steps + 1):
        state, laws, metrics = apply_laws(state, step, provenance,
                                          novelty_limit=req.novelty_limit,
                                          coherence_floor=req.coherence_floor)
        laws_union.extend(laws)
        if step % max(1, req.snapshot_every) == 0 or step == req.steps:
            snapshots.append(SimSnapshot(step=step, state=state, provenance=list(provenance), metrics=metrics))
    METRICS["sim_runs"] += 1
    record("sim.run", {"steps": req.steps, "laws": list(dict.fromkeys(laws_union))})
    latency = int((time.time() - start) * 1000)
    return state, snapshots, list(dict.fromkeys(laws_union))

# ---------------------------
# Middlewares and hooks
# ---------------------------

@app.middleware("http")
async def metrics_mw(request: Request, call_next):
    METRICS["requests_total"] += 1
    try:
        response = await call_next(request)
        return response
    except Exception as e:
        METRICS["errors_total"] += 1
        log.exception("Unhandled error")
        return JSONResponse({"error": str(e)}, status_code=500)

# ---------------------------
# HTML dashboard template
# ---------------------------

DASHBOARD_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>Multimodal AI + Γ-Universe</title>
<link rel="preconnect" href="https://fonts.gstatic.com" />
<link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@600;700&family=Open+Sans:wght@400;600&display=swap" rel="stylesheet" />
<style>
  :root {
    --bg1: #0f1020; --bg2: #141834;
    --fg: #eaeaf2; --muted: #aab; --accent: #7c4dff; --accent2: #24c8db;
  }
  * { box-sizing: border-box; }
  body {
    margin: 0; font-family: "Open Sans", system-ui, -apple-system, Segoe UI, Roboto, sans-serif;
    color: var(--fg); background: linear-gradient(135deg, var(--bg1), var(--bg2));
  }
  header {
    padding: 24px; display: flex; align-items: baseline; justify-content: space-between;
  }
  .brand { font-family: "Montserrat"; font-weight: 700; font-size: 20px; letter-spacing: 0.5px; }
  .grid {
    display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 16px; padding: 16px; max-width: 1200px; margin: 0 auto;
  }
  .card {
    background: rgba(255,255,255,0.05); border: 1px solid rgba(255,255,255,0.08);
    border-radius: 12px; padding: 16px; backdrop-filter: blur(4px);
  }
  .chip { display: inline-block; padding: 4px 8px; border-radius: 999px; font-size: 12px; border: 1px solid rgba(255,255,255,0.2); margin-right: 6px;}
  button, .btn {
    background: linear-gradient(90deg, var(--accent), var(--accent2)); color: #fff; border: none; padding: 10px 14px; border-radius: 8px; font-weight: 600; cursor: pointer;
  }
  input, textarea, select {
    width: 100%; padding: 10px; border-radius: 8px; background: rgba(255,255,255,0.06); color: var(--fg); border: 1px solid rgba(255,255,255,0.12); outline: none;
  }
  .muted { color: var(--muted); font-size: 12px; }
  .row { display: flex; gap: 10px; align-items: center; flex-wrap: wrap; }
  .title { font-family: "Montserrat"; font-weight: 700; font-size: 16px; margin-bottom: 8px; }
  .mono { font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, "Liberation Mono", monospace; }
  .kv { display: grid; grid-template-columns: auto 1fr; gap: 6px 12px; font-size: 13px; }
</style>
</head>
<body>
<header>
  <div class="brand">Multimodal AI + Γ-Universe</div>
  <div>
    <a class="chip" href="/auth/google" title="Google">Google</a>
    <a class="chip" href="/auth/github" title="GitHub">GitHub</a>
    <a class="chip" href="/auth/microsoft" title="Microsoft">Microsoft</a>
    <a class="chip" href="/logout">Logout</a>
  </div>
</header>

<div class="grid">
  <div class="card">
    <div class="title">System status</div>
    <div class="kv" id="status"></div>
  </div>

  <div class="card">
    <div class="title">Activity</div>
    <div id="activity" class="mono" style="font-size:12px; max-height: 220px; overflow:auto;"></div>
  </div>

  <div class="card">
    <div class="title">NLP playground</div>
    <div class="row">
      <select id="nlpTask">
        <option value="summarize">summarize</option>
        <option value="generate">generate</option>
        <option value="translate">translate</option>
        <option value="classify">classify</option>
      </select>
      <button onclick="runNLP()">Run</button>
    </div>
    <textarea id="nlpInput" rows="5" placeholder="Enter text..."></textarea>
    <div class="muted">Output</div>
    <pre class="mono" id="nlpOut" style="white-space:pre-wrap;"></pre>
  </div>

  <div class="card">
    <div class="title">Vision detection</div>
    <input id="visionFile" type="file" accept="image/*" />
    <div class="row"><button onclick="runVision()">Detect</button></div>
    <pre class="mono" id="visionOut" style="white-space:pre-wrap;"></pre>
  </div>

  <div class="card">
    <div class="title">Speech transcription</div>
    <input id="speechFile" type="file" accept="audio/*" />
    <div class="row"><button onclick="runSpeech()">Transcribe</button></div>
    <pre class="mono" id="speechOut" style="white-space:pre-wrap;"></pre>
  </div>

  <div class="card">
    <div class="title">Γ-universe simulation</div>
    <textarea id="seed" rows="5" class="mono">{ "energy": 1.0, "mode": "proto", "memo": "seed" }</textarea>
    <div class="row">
      <input id="steps" type="number" value="10" />
      <input id="novelty" type="number" step="0.05" value="0.3" />
      <input id="coherence" type="number" step="0.05" value="0.5" />
      <button onclick="runSim()">Simulate</button>    </div>
    <pre class="mono" id="simOut" style="white-space:pre-wrap; max-height:200px; overflow:auto;"></pre>
  </div>
</div>

<script>
async function fetchJSON(url) {
  const res = await fetch(url);
  return await res.json();
}
function kv(el, data) {
  el.innerHTML = Object.entries(data).map(([k,v]) => <div class="muted">${k}</div><div>${typeof v==="number"? v.toFixed? v.toFixed(2):v : v}</div>).join("");
}
async function refresh() {
  const h = await fetchJSON("/health");
  kv(document.getElementById("status"), h);
  const acts = await fetchJSON("/api/activity");
  document.getElementById("activity").innerText = acts.map(a => ${a.ts}  ${a.event}  ${JSON.stringify(a.detail)}).reverse().slice(0,50).join("\\n");
}
setInterval(refresh, 1500); refresh();

async function runNLP() {
  const input_text = document.getElementById("nlpInput").value;
  const task = document.getElementById("nlpTask").value;
  const res = await fetch("/api/nlp", { method: "POST", headers: {"Content-Type":"application/json"}, body: JSON.stringify({task, input_text})});
  document.getElementById("nlpOut").innerText = JSON.stringify(await res.json(), null, 2);
}

async function runVision() {
  const file = document.getElementById("visionFile").files[0];
  if (!file) return alert("Choose an image");
  const fd = new FormData();
  fd.append("image", file);
  const res = await fetch("/api/vision", {method:"POST", body: fd});
  document.getElementById("visionOut").innerText = JSON.stringify(await res.json(), null, 2);
}

async function runSpeech() {
  const file = document.getElementById("speechFile").files[0];
  if (!file) return alert("Choose audio");
  const fd = new FormData();
  fd.append("audio", file);
  const res = await fetch("/api/speech", {method:"POST", body: fd});
  document.getElementById("speechOut").innerText = JSON.stringify(await res.json(), null, 2);
}

async function runSim() {
  let seed;
  try { seed = JSON.parse(document.getElementById("seed").value); } catch(e) { return alert("Invalid seed JSON"); }
  const steps = parseInt(document.getElementById("steps").value, 10);
  const novelty = parseFloat(document.getElementById("novelty").value);
  const coherence = parseFloat(document.getElementById("coherence").value);
  const res = await fetch("/api/simulate", { method:"POST", headers: {"Content-Type":"application/json"}, body: JSON.stringify({seed, steps, noveltylimit:novelty, coherencefloor:coherence, snapshot_every: Math.max(1, Math.floor(steps/2))}) });
  document.getElementById("simOut").innerText = JSON.stringify(await res.json(), null, 2);
}
</script>
</body>
</html>
"""

# ---------------------------
# Routes
# ---------------------------

@app.get("/", response_class=HTMLResponse)
async def dashboard(request: Request):
    return HTMLResponse(DASHBOARD_HTML)

@app.get("/health", response_model=Health)
async def health():
    uptime = time.time() - METRICS["start_time"]
    return Health(status="ok", uptimeseconds=uptime, demomode=settings.DEMO_MODE, version="1.0.0")

@app.get("/api/activity")
async def activity():
    return ACTIVITY[-200:]

OAuth routes

@app.get("/auth/{provider}")
async def oauth_start(provider: str, request: Request):
    if provider not in OAUTH_PROVIDERS:
        raise HTTPException(404, "Unknown provider")
    if not oauth_enabled(provider):
        record("auth.demo-login", {"provider": provider})
        # Demo mode: create a synthetic user
        request.session["user"] = {"name": "Demo User", "provider": provider, "sub": f"demo-{provider}-{secrets.token_hex(4)}"}
        return RedirectResponse("/")
    cfg = OAUTH_PROVIDERS[provider]
    state = secrets.token_urlsafe(16)
    request.session["oauth_state"] = state
    redirecturi = f"{settings.OAUTHREDIRECT_BASE}/auth/{provider}/callback"
    params = {
        "clientid": cfg["clientid"],
        "redirecturi": redirecturi,
        "response_type": "code",
        "scope": cfg["scope"],
        "state": state,
    }
    return RedirectResponse(f"{cfg['auth_url']}?{urlencode(params)}")

@app.get("/auth/{provider}/callback")
async def oauth_callback(provider: str, request: Request, code: Optional[str] = None, state: Optional[str] = None):
    if provider not in OAUTH_PROVIDERS:
        raise HTTPException(404, "Unknown provider")
    expect = request.session.get("oauth_state")
    request.session.pop("oauth_state", None)
    if not code or not state or state != expect:
        raise HTTPException(400, "Invalid OAuth state")
    cfg = OAUTH_PROVIDERS[provider]
    redirecturi = f"{settings.OAUTHREDIRECT_BASE}/auth/{provider}/callback"
    data = {
        "clientid": cfg["clientid"], "clientsecret": cfg["clientsecret"],
        "code": code, "redirecturi": redirecturi, "granttype": "authorizationcode",
    }
    headers = {"Accept": "application/json"}
    async with httpx.AsyncClient(timeout=30) as client:
        tokenres = await client.post(cfg["tokenurl"], data=data, headers=headers)
        tokenres.raisefor_status()
        token = token_res.json()
        accesstoken = token.get("accesstoken")
        if not access_token:
            raise HTTPException(400, "No access token")
        uiheaders = {"Authorization": f"Bearer {accesstoken}"}
        userinfores = await client.get(cfg["userinfourl"], headers=ui_headers)
        userinfores.raisefor_status()
        profile = userinfo_res.json()
    # Normalize
    user = {
        "provider": provider,
        "sub": profile.get("sub") or str(profile.get("id")),
        "name": profile.get("name") or profile.get("login") or profile.get("email") or "user",
        "email": profile.get("email"),
        "avatar": profile.get("picture") or profile.get("avatar_url"),
    }
    request.session["user"] = user
    record("auth.login", {"provider": provider, "sub": user["sub"]}, user=user["name"])
    return RedirectResponse("/")

@app.get("/logout")
async def logout(request: Request):
    request.session.pop("user", None)
    return RedirectResponse("/")

NLP

@app.post("/api/nlp", response_model=NLPResponse)
async def api_nlp(req: NLPRequest, request: Request):
    start = time.time()
    METRICS["nlp_calls"] += 1
    task = req.task.lower()
    text = req.input_text
    out = ""
    model_used = "demo"
    if not settings.DEMO_MODE:
        try:
            pipes = load_nlp()
            if task == "summarize":
                out = pipes"summarize"[0]["summarytext"]
                modelused = settings.NLPMODEL
            elif task == "generate":
                out = pipes"generate"[0]["generated_text"]
                modelused = settings.NLPMODEL
            elif task == "translate":
                out = pipes"translate"[0]["translationtext"]
                model_used = "t5-small en->de"
            elif task == "classify":
                out = pipes"classify"[0]["label"]
                model_used = "distilbert-sst2"
            else:
                raise HTTPException(400, "Unknown task")
        except Exception as e:
            log.warning(f"NLP fallback: {e}")
            settings.DEMO_MODE = True  # soft flip to demo on failure
    if settings.DEMO_MODE:
        if task == "summarize":
            out = demo_summarize(text)
        elif task == "generate":
            out = demo_generate(text)
        elif task == "translate":
            out = demotranslateen_de(text)
        elif task == "classify":
            out = demo_classify(text)
        else:
            raise HTTPException(400, "Unknown task")
    latency = int((time.time() - start) * 1000)
    record("nlp.run", {"task": task, "latency_ms": latency})
    return NLPResponse(task=task, outputtext=out, model=modelused, demo=settings.DEMOMODE, latencyms=latency)

Vision

@app.post("/api/vision", response_model=VisionResponse)
async def api_vision(image: UploadFile = File(...)):
    start = time.time()
    METRICS["vision_calls"] += 1
    content = await image.read()
    if Image is None or np is None:
        raise HTTPException(500, "Pillow/numpy not installed")
    img = Image.open(io.BytesIO(content)).convert("RGB")
    model_used = "demo"
    detections: List[Dict[str, Any]] = []
    if not settings.DEMO_MODE:
        try:
            yolo = load_yolo()
            res = yolo.predict(img, verbose=False)[0]
            for b in res.boxes:
                xyxy = b.xyxy[0].tolist()
                conf = float(b.conf[0])
                cls_idx = int(b.cls[0])
                label = yolo.names.get(clsidx, str(clsidx))
                detections.append({"label": label, "confidence": conf, "box": [float(x) for x in xyxy]})
            modelused = settings.YOLOMODEL
        except Exception as e:
            log.warning(f"YOLO fallback: {e}")
            settings.DEMO_MODE = True
    if settings.DEMO_MODE:
        detections = demo_yolo(img)
    latency = int((time.time() - start) * 1000)
    record("vision.run", {"latency_ms": latency, "count": len(detections)})
    return VisionResponse(detections=detections, model=modelused, demo=settings.DEMOMODE, latency_ms=latency)

Speech

@app.post("/api/speech", response_model=SpeechResponse)
async def api_speech(audio: UploadFile = File(...)):
    start = time.time()
    METRICS["speech_calls"] += 1
    content = await audio.read()
    model_used = "demo"
    text = ""
    if not settings.DEMO_MODE:
        try:
            whisper = load_whisper()
            # Write to tmp buffer for whisper
            tmppath = os.path.join(settings.BLOBDIR, f"tmp-{secrets.token_hex(8)}.wav")
            with open(tmp_path, "wb") as f:
                f.write(content)
            result = whisper.transcribe(tmp_path)
            os.remove(tmp_path)
            text = result.get("text", "")
            modelused = settings.WHISPERMODEL
        except Exception as e:
            log.warning(f"Whisper fallback: {e}")
            settings.DEMO_MODE = True
    if settings.DEMO_MODE:
        text = demowhisperdummy()
    latency = int((time.time() - start) * 1000)
    record("speech.run", {"latency_ms": latency})
    return SpeechResponse(text=text, model=modelused, demo=settings.DEMOMODE, latency_ms=latency)

Blob storage

@app.post("/api/blob/upload", response_model=BlobUploadResponse)
async def blob_upload(file: UploadFile = File(...)):
    if not filesizeok(file):
        raise HTTPException(413, "File too large")
    data = await file.read()
    name = f"{int(time.time())}-{secrets.token_hex(8)}-{file.filename}"
    path = os.path.join(settings.BLOB_DIR, name)
    with open(path, "wb") as f:
        f.write(data)
    token = signer.dumps({"name": name, "ts": time.time()})
    url = f"/api/blob/get?token={token}"
    METRICS["blob_uploads"] += 1
    record("blob.upload", {"name": name, "bytes": len(data)})
    return BlobUploadResponse(name=name, size=len(data), url=url, demo=settings.DEMO_MODE)

@app.get("/api/blob/get")
async def blob_get(token: str):
    try:
        payload = signer.loads(token, max_age=3600 * 24)
        name = payload["name"]
    except Exception:
        raise HTTPException(400, "Invalid token")
    path = os.path.join(settings.BLOB_DIR, name)
    if not os.path.exists(path):
        raise HTTPException(404, "Not found")
    with open(path, "rb") as f:
        data = f.read()
    return Response(content=data, media_type="application/octet-stream",
                    headers={"Content-Disposition": f'attachment; filename="{os.path.basename(path)}"'})

Simulation

@app.post("/api/simulate", response_model=SimResponse)
async def api_simulate(req: SimRequest):
    start = time.time()
    finalstate, snaps, laws = runsimulation(req)
    latency = int((time.time() - start) * 1000)
    return SimResponse(finalstate=finalstate, snapshots=snaps, lawsapplied=laws, demo=settings.DEMOMODE, latency_ms=latency)

Metrics detail

@app.get("/metrics")
async def metrics():
    up = time.time() - METRICS["start_time"]
    out = dict(METRICS)
    out["uptime_seconds"] = up
    return out

# ---------------------------
# Typed client
# ---------------------------

class TypedClient:
    def init(self, base_url: str, timeout: float = 30.0, token: Optional[str] = None):
        self.baseurl = baseurl.rstrip("/")
        self.timeout = timeout
        self.token = token

    def _headers(self):
        h = {"Accept": "application/json"}
        if self.token:
            h["Authorization"] = f"Bearer {self.token}"
        return h

    async def health(self) -> Health:
        async with httpx.AsyncClient(timeout=self.timeout) as client:
            r = await client.get(f"{self.baseurl}/health", headers=self.headers())
            r.raiseforstatus()
            return Health(r.json())

    async def nlp(self, task: str, input_text: str) -> NLPResponse:
        async with httpx.AsyncClient(timeout=self.timeout) as client:
            r = await client.post(f"{self.baseurl}/api/nlp", json={"task": task, "inputtext": inputtext}, headers=self.headers())
            r.raiseforstatus()
            return NLPResponse(r.json())

    async def vision(self, image_bytes: bytes, filename: str = "image.png") -> VisionResponse:
        files = {"image": (filename, image_bytes, "application/octet-stream")}
        async with httpx.AsyncClient(timeout=self.timeout) as client:
            r = await client.post(f"{self.baseurl}/api/vision", files=files, headers=self.headers())
            r.raiseforstatus()
            return VisionResponse(r.json())

    async def speech(self, audio_bytes: bytes, filename: str = "audio.wav") -> SpeechResponse:
        files = {"audio": (filename, audio_bytes, "application/octet-stream")}
        async with httpx.AsyncClient(timeout=self.timeout) as client:
            r = await client.post(f"{self.baseurl}/api/speech", files=files, headers=self.headers())
            r.raiseforstatus()
            return SpeechResponse(r.json())

    async def simulate(self, seed: Dict[str, Any], steps: int = 10) -> SimResponse:
        payload = {"seed": seed, "steps": steps}
        async with httpx.AsyncClient(timeout=self.timeout) as client:
            r = await client.post(f"{self.baseurl}/api/simulate", json=payload, headers=self.headers())
            r.raiseforstatus()
            return SimResponse(r.json())

# ---------------------------
# Entrypoint
# ---------------------------

if name == "main":
    import uvicorn
    host = os.getenv("HOST", "127.0.0.1")
    port = int(os.getenv("PORT", "8000"))
    log.info(f"Starting server on http://{host}:{port}  demo={settings.DEMO_MODE}")
    uvicorn.run("app_single:app", host=host, port=port, reload=False)